In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Carregando os dados do arquivo Excel
file_path = 'latencies by service.xlsx'
df = pd.read_excel(file_path)

# Convertendo a coluna "latencies" de string para uma lista de números
df['latencies'] = df['latencies'].apply(lambda x: [int(n) for n in x.split(',')])

# Calculando estatísticas descritivas para cada registro de hora
descriptive_stats = df['latencies'].apply(lambda x: pd.Series({
    'mean': np.mean(x),
    'median': np.median(x),
    'std_dev': np.std(x),
    'min': np.min(x),
    'max': np.max(x),
    '25th_percentile': np.percentile(x, 25),
    '75th_percentile': np.percentile(x, 75)
}))

# Definindo um limite para identificar anomalias (valores acima do 75º percentil + 1.5 vezes o IQR)
descriptive_stats['IQR'] = descriptive_stats['75th_percentile'] - descriptive_stats['25th_percentile']
threshold = descriptive_stats['75th_percentile'] + 1.5 * descriptive_stats['IQR']

# Função para identificar anomalias em cada lista de latências
def find_anomalies(latencies, threshold):
    return [x for x in latencies if x > threshold]

# Aplicando a função para encontrar anomalias
df['anomalies'] = df.apply(lambda row: find_anomalies(row['latencies'], threshold.loc[row.name]), axis=1)

# Preparando os dados para o gráfico
times = []
anomalies = []
for _, row in df.iterrows():
    times.extend(row['latencies'])
    anomalies.extend(row['anomalies'])

anomaly_indicators = ["Anomalia" if time in anomalies else "Normal" for time in times]

visualization_df = pd.DataFrame({
    'Tempo de Resposta': times,
    'Tipo': anomaly_indicators
})

# Criando o gráfico
plt.figure(figsize=(12, 6))
sns.boxplot(x='Tempo de Resposta', y='Tipo', data=visualization_df, palette="Set2")
plt.title("Distribuição do Tempo de Resposta com Anomalias Identificadas")
plt.xlabel("Tempo de Resposta (ms)")
plt.ylabel("Tipo")
plt.show()
